# Reading, Writing and Validating Data in PySpark HW

Welcome to your first coding homework assignment in PySpark! I hope you enjoyed the lecture on Reading, Writing and Validating dataframes. Now it's time to put what you've learned into action! 

I've included several instructions below to help guide you through this homework assignment which I hope will get you feeling even comfortable reading, writing and validating dataframes. If you get stuck at any point, feel free to jump to the next lecture where I will guide you through my solutions to the HW assignment. 

Have fun!

Let's dig right in!


## But first things first.....
We need to always begin every Spark session by creating a Spark instance. Let's go ahead and use the method we learned in the lecture in the cell below. Also see if you can remember how to open the Spark UI (using a link that automatically guides you there). 

In [3]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('FirstHM').getOrCreate()
spark

## Next let's start by reading a basic csv dataset

Download the pga_tour_historical dataset that is attached to this lecture and save it whatever folder you want, then read it in. 

**Data Source:** https://www.kaggle.com/bradklassen/pga-tour-20102018-data

Rememer to try letting Spark infer the header and infer the Schema types!

In [4]:
spark_df = spark.read.csv('datasets/pga_tour_historical.csv', header=True, inferSchema=True)

## 1. View first 5 lines of dataframe
First generate a view of the first 5 lines of the dataframe to get an idea of what is inside. We went over two ways of doing this... see if you can remember BOTH ways. 

In [5]:
spark_df.show(5)

+---------------+------+----------------+--------------------+-----+
|    Player Name|Season|       Statistic|            Variable|Value|
+---------------+------+----------------+--------------------+-----+
|Robert Garrigus|  2010|Driving Distance|Driving Distance ...|   71|
|   Bubba Watson|  2010|Driving Distance|Driving Distance ...|   77|
| Dustin Johnson|  2010|Driving Distance|Driving Distance ...|   83|
|Brett Wetterich|  2010|Driving Distance|Driving Distance ...|   54|
|    J.B. Holmes|  2010|Driving Distance|Driving Distance ...|  100|
+---------------+------+----------------+--------------------+-----+
only showing top 5 rows



## 2. Print the schema details

Now print the details of the dataframes schema that Spark infered to ensure that it was infered correctly. Sometimes it is not infered correctly, so we need to watch out!

In [8]:
spark_df.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: string (nullable = true)



## 3. Edit the schema during the read in

We can see from the output above that Spark did not correctly infer that the "value" column was an integer value. Let's try specifying the schema this time to let spark know what the schema should be.

Here is a link to see a list of PySpark data types in case you need it (also attached to the lecture): 
https://spark.apache.org/docs/latest/sql-ref-datatypes.html

In [12]:
from pyspark.sql.types import *

data_schema = [StructField("Player Name", StringType(), True),
              StructField("Season", IntegerType(), True),
              StructField("Statistic", StringType(), True),
              StructField("Variable", StringType(), True),
              StructField("Value", IntegerType(), True)]

df = spark.read.csv('datasets/pga_tour_historical.csv', schema=StructType(fields=data_schema))
df.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: integer (nullable = true)



## 4. Generate summary statistics for only one variable

See if you can generate summary statistics for only the "Value" column using the .describe function

(count, mean, stddev, min, max) 

In [14]:
df.describe("Value").summary("count", "mean", "stddev", "min", "max").show()

+-------+-------+------------------+
|summary|summary|             Value|
+-------+-------+------------------+
|  count|      5|                 5|
|   mean|   null|  1078358.42914689|
| stddev|   null|1554507.2871420535|
|    min|  count|              -178|
|    max| stddev|           3564954|
+-------+-------+------------------+



## 5. Generate summary statistics for TWO variables
Now try to generate ONLY the count min and max for BOTH the "Value" and "Season" variable using the select. You can't use the .describe function for this one but see if you can remember which function you CAN use. 

In [16]:
df.select("Value", "Season").summary("min", "max").show()

+-------+-------+------+
|summary|  Value|Season|
+-------+-------+------+
|    min|   -178|  2010|
|    max|3564954|  2018|
+-------+-------+------+



## 6. Write a parquet file

Now try writing a parquet file (not partitioned) from the pga dataset. But first create a new dataframe containing ONLY the the "Season" and "Value" fields (using the "select command you used in the question above) and write a parquet file partitioned by "Season". This is a bit of a challenge aimed at getting you ready for material that will be covered later on in the course. Don't feel bad if you can't figure it out.

*Note that if any of your variable names contain spaces, spark will produce an error message with this call. That is why we are selecting ONLY the "Season" and "Value" fields. Ideally we should renamed those columns but we haven't gotten to that yet in this course but we will soon!*

In [18]:
season_value = df.select("Value", "Season")
season_value.write.mode("overwrite").parquet("to_parquet")

## 7. Write a partioned parquet file

You will need to use the same limited dataframe that you created in the previous question to accomplish this task as well. 

In [19]:
season_value.write.mode("overwrite").partitionBy("Season").parquet("part_parquet")

## 8. Read in a partitioned parquet file

Now try reading in the partitioned parquet file you just created above. 

In [23]:
df = spark.read.parquet("to_parquet")
df.show()

+-----+------+
|Value|Season|
+-----+------+
|  992|  2016|
| null|  2016|
|  978|  2016|
|  794|  2016|
|  822|  2016|
| null|  2016|
|  698|  2016|
|  990|  2016|
|  728|  2016|
|  616|  2016|
|  896|  2016|
| null|  2016|
|  906|  2016|
|  880|  2016|
|  906|  2016|
| null|  2016|
|  668|  2016|
| null|  2016|
|  810|  2016|
|  542|  2016|
+-----+------+
only showing top 20 rows



## 9. Reading in a set of paritioned parquet files

Now try only reading Seasons 2010, 2011 and 2012.

In [22]:
df = spark.read.parquet("part_parquet/Season=2010", "part_parquet/Season=2011", "part_parquet/Season=2012")
df.show()

+-----+
|Value|
+-----+
|   71|
|   77|
|   83|
|   54|
|  100|
|   63|
|   88|
|   64|
|   64|
|   92|
|   75|
|   54|
|   76|
|   94|
|   82|
|   85|
|   79|
|   89|
|   88|
|   91|
+-----+
only showing top 20 rows



## 10. Create your own dataframe

Try creating your own dataframe below using PySparks *.createDataFrame* function. See if you can make one that contains 4 variables and at least 3 rows. 

Let's see how creative you can get on the content of the dataframe :)

In [25]:
values = [("cheese", 1), ("eggs", 12), ("limons", 3)]
df = spark.createDataFrame(values, ["supermarket_list", "amount"])
df.show()

+----------------+------+
|supermarket_list|amount|
+----------------+------+
|          cheese|     1|
|            eggs|    12|
|          limons|     3|
+----------------+------+



## We're done! Great job!